### Standalone SNOWSTORM SNOMED access example

In [1]:
import time
import random
import requests

# Public training Snowstorm FHIR endpoint (recommended for class exercises)
SNOWSTORM_FHIR_BASE = "https://snowstorm-training.snomedtools.org/fhir"  # :contentReference[oaicite:2]{index=2}

SNOMED_SYSTEM = "http://snomed.info/sct"

# If you ever need auth (usually you won't on the public training endpoint), set token here:
SNOWSTORM_BEARER_TOKEN = ""  # e.g., "eyJhbGciOi..."

def _headers():
    h = {
        "Accept": "application/fhir+json",
        "User-Agent": "DATASCI290-Snowstorm-Client/1.0"
    }
    if SNOWSTORM_BEARER_TOKEN:
        h["Authorization"] = f"Bearer {SNOWSTORM_BEARER_TOKEN}"
    return h

def fhir_get(path: str, params=None, max_retries: int = 8):
    """
    Robust GET that handles 429 rate limits by respecting Retry-After when present.
    """
    url = SNOWSTORM_FHIR_BASE.rstrip("/") + "/" + path.lstrip("/")
    backoff = 1.0
    for attempt in range(max_retries):
        r = requests.get(url, headers=_headers(), params=params, timeout=60)

        if r.status_code == 429:
            # Respect server hint if provided
            retry_after = r.headers.get("Retry-After")
            if retry_after is not None:
                try:
                    sleep_s = float(retry_after)
                except ValueError:
                    sleep_s = backoff
            else:
                sleep_s = backoff

            # Add small jitter so a whole class doesn't synchronize
            sleep_s = sleep_s + random.uniform(0, 0.25)
            print(f"[429] Rate limited. Sleeping {sleep_s:.2f}s then retrying (attempt {attempt+1}/{max_retries})")
            time.sleep(sleep_s)
            backoff = min(backoff * 2, 30.0)
            continue

        # Other errors: raise with context
        r.raise_for_status()
        return r.json()

    raise RuntimeError("Exceeded retry budget due to repeated 429 rate limits.")

# 0) Check server
cap = fhir_get("metadata")
print("FHIR server OK. Software:", cap.get("software", {}).get("name"), cap.get("software", {}).get("version"))

# 1) Lookup a known SCTID (example: Chest pain is commonly 29857009)
def snomed_lookup(sctid: str) -> str:
    out = fhir_get("CodeSystem/$lookup", params={"system": SNOMED_SYSTEM, "code": sctid})
    if "display" in out:
        return out["display"]
    for p in out.get("parameter", []):
        if p.get("name") == "display":
            return p.get("valueString", "")
    return ""

print("Lookup 29857009:", snomed_lookup("29857009"))

# 2) Text filter expansion (may or may not be enabled on a given server)
def snomed_suggest(text: str, count: int = 10):
    params = {
        "url": f"{SNOMED_SYSTEM}?fhir_vs",
        "filter": text,
        "count": count
    }
    vs = fhir_get("ValueSet/$expand", params=params)
    contains = vs.get("expansion", {}).get("contains", [])
    return [(c.get("code"), c.get("display")) for c in contains]

try:
    print("\nSuggestions for 'shortness of breath':")
    for code, disp in snomed_suggest("shortness of breath", count=10):
        print(" ", code, "-", disp)
except requests.HTTPError as e:
    print("\nThis server did not accept filter-based $expand for suggestions.")
    print("HTTP:", e)

# 3) ECL expansion (very useful for building a scoped subset)
def snomed_expand_ecl(ecl: str, count: int = 25):
    params = {
        "url": f"{SNOMED_SYSTEM}?fhir_vs=ecl/{ecl}",
        "count": count
    }
    vs = fhir_get("ValueSet/$expand", params=params)
    contains = vs.get("expansion", {}).get("contains", [])
    return [(c.get("code"), c.get("display")) for c in contains]

ecl = "<< 29857009"   # descendants-or-self of Chest pain
expanded = snomed_expand_ecl(ecl, count=25)
print(f"\nECL expand {ecl} (first {len(expanded)}):")
for code, disp in expanded:
    print(" ", code, "-", disp)


FHIR server OK. Software: Snowstorm FHIR Server 4.12.0
Lookup 29857009: Chest pain

Suggestions for 'shortness of breath':
  267036007 - SOB - Shortness of breath
  135833008 - CLASP shortness of breath score
  161941007 - SOBAR - Shortness of breath at rest
  60845006 - Dyspnea on effort
  763074005 - Cardiovascular Limitations and Symptoms Profile shortness of breath score
  767499000 - Autosomal recessive congenital methaemoglobinaemia type I
  764955006 - Ventricular septal defect with aortic insufficiency

ECL expand << 29857009 (first 25):
  16754391000119100 - Stable angina due to coronary arteriosclerosis
  15960661000119107 - Arteriosclerosis of coronary artery bypass graft with unstable angina
  15960581000119102 - Angina co-occurrent and due to arteriosclerosis of autologous vein coronary artery bypass graft
  15960541000119107 - Unstable angina due to arteriosclerosis of autologous vein coronary artery bypass graft
  15960381000119109 - Angina co-occurrent and due to arteri